In [2]:
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MaxAbsScaler
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib_inline.backend_inline
import seaborn as sns
import matplotlib as mpl
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')
from scipy import stats
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.impute import KNNImputer
import time

from tabpfn import TabPFNClassifier

# Data preparation for model
- Imputing NA values
- Expand features
- Transform numerical features
- Split train/validation data
- Scaling

In [3]:
data = np.load('../data/train_curated.npz',allow_pickle=True)
X = data['train']
Y = data['target']
feats_names = data['feats']

data = np.load('../data/test_curated.npz',allow_pickle=True)
X_test = data['test']
LABELS_SUB = data['labels']
print(X.shape)
print(X_test.shape)

(8693, 27)
(4277, 27)


In [4]:
all_data_train = True

x_train, x_cv, y_train, y_cv = train_test_split(X,Y,test_size=0.3)

if all_data_train:
    x_train = X
    y_train = Y

x_train.shape

(8693, 27)

In [5]:
scaler = MaxAbsScaler().fit(x_train)
x_train = scaler.transform(x_train)
x_cv = scaler.transform(x_cv)
X_test = scaler.transform(X_test)

In [6]:
classifier = TabPFNClassifier(N_ensemble_configurations=5)

In [7]:
start = time.time()
classifier.fit(x_train, y_train,overwrite_warning=True)
y_eval, p_eval = classifier.predict(x_cv, return_winning_probability=True)
print('Prediction time: ', time.time() - start, 'Accuracy', accuracy_score(y_cv, y_eval))

c:\Users\artur\Desktop\lneoc\pibic_lneoc\deep_learning\venv\Lib\site-packages\torch\utils\checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Prediction time:  867.0846319198608 Accuracy 0.8182515337423313


In [8]:
x_train = X
y_train = Y
y_test = classifier.predict(X_test)


c:\Users\artur\Desktop\lneoc\pibic_lneoc\deep_learning\venv\Lib\site-packages\torch\utils\checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


# Exporting for submission

In [11]:
print(y_test)
predictions = pd.DataFrame(y_test,columns=['Transported'])
predictions['PassengerId'] = LABELS_SUB
predictions['Transported'] = predictions['Transported'].astype(bool)
predictions = predictions[['PassengerId','Transported']]

[ True False  True ...  True  True  True]


In [12]:
predictions.to_csv('../data/submissionTABPFN_scaled.csv',index=False)